In [1]:
import sys
sys.path.append('../')
import mblib
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import tree


random_state=42
np.random.seed(random_state)


df = pd.read_csv('dataset.csv', low_memory=False)
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)

mblib.display_information_dataframe(df,True,True,True)

ModuleNotFoundError: No module named 'mblib'

In [ ]:
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df)

categorical_columns = []
for col in df.columns[df.dtypes == object]:
    if col != "Attack_type":
        categorical_columns.append(col)

featuresFromStart = [ col for col in df.columns if col not in ["Attack_label"]+["Attack_type"]]

colunas_one_hot = {}
for coluna in categorical_columns:
    codes, uniques = pd.factorize(df[coluna].unique())
    colunas_one_hot[coluna] = {"uniques": uniques, "codes":codes}
    df[coluna] = df[coluna].replace(colunas_one_hot[coluna]["uniques"], colunas_one_hot[coluna]["codes"])
    print(coluna)
df = pd.get_dummies(data=df, columns=categorical_columns)


http.request.method
http.referer
http.request.version
dns.qry.name.len
mqtt.conack.flags
mqtt.protoname
mqtt.topic


In [ ]:

features = [ col for col in df.columns if col not in ["Attack_label"]+["Attack_type"]]
n_total = len(df)

train_val_indices, test_indices = train_test_split(range(n_total), test_size=0.2, random_state=random_state)

X_train = df[features].values[train_val_indices]
y_train = df["Attack_label"].values[train_val_indices]

X_test = df[features].values[test_indices]
y_test = df["Attack_label"].values[test_indices]

standScaler = StandardScaler()
model_norm = standScaler.fit(X_train)

X_train = model_norm.transform(X_train)
X_test = model_norm.transform(X_test)
#X_valid = model_norm.transform(X_valid)

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=20, random_state = random_state)#,n_estimators = 10)
# Train the model on training data
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

In [ ]:
len(df.index)

13893

In [ ]:
mblib.calculate_metrics("Decision tree", y_test, predictions, average='micro')

--- Performance of Decision tree ---
Accuracy : 100.0%
Precision: 100.0%
Recall: 100.0%
F1-score: 100.0%
Balanced accuracy: 100.0%
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       185
           1       1.00      1.00      1.00      2594

    accuracy                           1.00      2779
   macro avg       1.00      1.00      1.00      2779
weighted avg       1.00      1.00      1.00      2779

